# Imports

In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Load the Pandas libraries with alias 'pd'
import datetime

import numpy as np
import pandas as pd
from bokeh.io import output_notebook
from bokeh.layouts import grid
from bokeh.models import BooleanFilter, CDSView, ColumnDataSource, LinearAxis, Range1d
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10_6, Category20_20
from bokeh.plotting import figure, show

from covid19.covid import (
    get_all_states_data,
    get_brazil_data,
    get_date_date_cases_greater_than,
    get_dia_de_contaminacao_array
)
from covid19.scrap import get_covid_data, read_datafile_from_disc

output_notebook(hide_banner=True)

import logging

logging.getLogger().setLevel(logging.INFO)

In [21]:
import qgrid

# Lendo os dados

In [7]:
# data = pd.read_excel("DT_PAINEL_COVIDBR_20200510.xlsx")
# data = read_datafile_from_disc("DT1_PAINEL_COVIDBR_20200512.xlsx")
data = get_covid_data()

# Entendendo os dados e separando dados do Brasil todo e de cada estado

As primeiras linhas possuem dados para o Brasil todo e a região é "Brasil".

In [26]:
print("Shape do dataset todo", data.shape)
qgrid.show_grid(data, grid_options=dict(maxVisibleRows=5))

Shape do dataset todo (75801, 15)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Note que a função que lê os dados adicionou uma coluna contendo o dia da semana.

Vamos separar o dataset em dois: um para o Brasil todo e outro para os estados. Note que colunas irrelevantes (como "município", por exemplo) foram removidas e 4 colunas novas foram adicionadas: **casosNovos**, **obitosNovos**, **diasDeContaminacao_1** e **diasDeContaminacao_100**.

In [30]:
data_brasil = get_brazil_data(data)
print("Shape do dataset do Brasil", data_brasil.shape)
qgrid.show_grid(data_brasil, grid_options=dict(maxVisibleRows=10))

Shape do dataset do Brasil (79, 12)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [10]:
data_estados = get_all_states_data(data)
print("Shape do dataset dos estados", data_estados.shape)
print(data_estados.shape[0] / 77)

Shape do dataset dos estados (1680, 15)
21.818181818181817


In [29]:
# data_estados.groupby(by="estado").count()
qgrid.show_grid(data_estados, grid_options=dict(maxVisibleRows=10))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Primeiros casos

In [12]:
# data_brasil = data.groupby("data").sum()

print(f"Data do primeiro caso: {get_date_date_cases_greater_than(data, 0)}")
print(
    f"Data com mais de 10 casos: {get_date_date_cases_greater_than(data, 10)}")
print(
    f"Data com mais de 100 casos: {get_date_date_cases_greater_than(data, 100)}"
)
print(
    f"Data com mais de 1000 casos: {get_date_date_cases_greater_than(data, 1000)}"
)

Data do primeiro caso: 2020-02-26
Data com mais de 10 casos: 2020-03-06
Data com mais de 100 casos: 2020-03-14
Data com mais de 1000 casos: 2020-03-21


In [34]:
qgrid.show_grid(data_estados[data_estados.estado=="CE"], grid_options=dict(maxVisibleRows=5))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Plot dados para o Brasil

In [17]:
source_brasil = ColumnDataSource(data_brasil)
fig = figure(title="Covid-19 no Brasil",
             x_axis_type='datetime',
             width=1200,
             height=400,
             x_axis_label="Data",
             y_axis_label="Casos Acumulados")
fig.line("data",
         "casosAcumulado",
         source=source_brasil,
         color=Category10_6[0],
         legend_label="Acumulados")
fig.circle("data",
           "casosAcumulado",
           source=source_brasil,
           color=Category10_6[0],
           legend_label="Acumulados")
fig.line("data",
         "obitosAcumulado",
         source=source_brasil,
         color=Category10_6[1],
         legend_label="Óbitos")
fig.circle("data",
           "obitosAcumulado",
           source=source_brasil,
           color=Category10_6[1],
           legend_label="Óbitos")
fig.legend.location = "top_left"
fig.legend.click_policy = 'hide'
fig.add_tools(
    HoverTool(
        tooltips=[("Casos Acumulados",
                   "@casosAcumulado"), ("Óbitos Acumulados", "@obitosAcumulado"
                                       ), ("Semana Epidemiológica",
                                           "@semanaEpi")]))

show(fig)

## Plot de vários estados por data

In [43]:
# state_abrv_vec = np.array(['CE', 'DF', 'PE', 'RJ', 'SP', 'BA', 'AM', 'MS', 'RN'])
state_abrv_vec = np.array(['CE', 'DF', 'RJ', 'SP', 'AM'])

def get_plot(column_name, y_axis_label="Label", width=1200, height=400):

    fig = figure(title="Covid-19 nos estados",
                 x_axis_type='datetime',
                 width=width,
                 height=height,
                 x_axis_label="Data",
                 y_axis_label=y_axis_label)
    for i, sigla in enumerate(state_abrv_vec):
        state_data = data_estados[data_estados.estado==sigla]
        source = ColumnDataSource(state_data)
        fig.line("data",
                 column_name,
                 source=source,
                 legend_label=sigla,
                 color=Category10_6[i])
        fig.circle("data",
                   column_name,
                   source=source,
                   legend_label=sigla,
                   color=Category10_6[i])

    fig.legend.location = "top_left"
    #     fig.legend.title = "Estado"
    fig.legend.click_policy = 'hide'
    fig.add_tools(
        HoverTool(
            tooltips=[("Estado", "@estado"), ("Dia da Semana", "@diaDaSemana"),
                      ("Casos Acumulados",
                       "@casosAcumulado"), ("Casos Novos", "@casosNovos"),
                      ("Óbitos Acumulados",
                       "@obitosAcumulado"), ("Óbitos Novos", "@obitosNovos")]))
    return fig


# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
width = 650
height = 300
plot_casosAcumulados = get_plot("casosAcumulado",
                                y_axis_label="Casos Acumulados",
                                width=width,
                                height=height)
plot_casosNovos = get_plot("casosNovos",
                           y_axis_label="Casos Novos",
                           width=width,
                           height=height)
plot_casosNovos.x_range = plot_casosAcumulados.x_range
plot_obitosAcumulados = get_plot("obitosAcumulado",
                                 y_axis_label="Óbitos Acumulados",
                                 width=width,
                                 height=height)
plot_obitosAcumulados.x_range = plot_casosAcumulados.x_range
plot_obitosNovos = get_plot("obitosNovos",
                            y_axis_label="Óbitos Novos",
                            width=width,
                            height=height)
plot_obitosNovos.x_range = plot_casosAcumulados.x_range

show(
    grid([[plot_casosAcumulados, plot_casosNovos],
          [plot_obitosAcumulados, plot_obitosNovos]]))

## Plot de vários estados por dia de contaminação


In [48]:
# Estados com crescimento mais rápido por dia de contaminação
estados_crescimento_rapido = ["CE", "SP", "AM", "PE", "RJ"]


def get_plot_por_dia_de_contaminacao(column_name,
                                     y_axis_label="Label",
                                     min_casos=1, # Deve ser ou 1 ou 100
                                     width=1200,
                                     height=400,
                                    title="Covid-19 nos 5 estados com crescimento mais rápido por dia de contaminação"):
    fig = figure(
        title=title,
        width=width,
        height=height,
        x_axis_label="Dia de Contaminação",
        y_axis_label=y_axis_label)

    for i, sigla in enumerate(estados_crescimento_rapido):
        state_data = data_estados[data_estados.estado==sigla]
        source = ColumnDataSource(state_data)
        fig.line(f"diasDeContaminacao_{min_casos}",
                 column_name,
                 source=source,
                 legend_label=sigla,
                 color=Category20_20[i % 20])
        fig.circle(f"diasDeContaminacao_{min_casos}",
                   column_name,
                   source=source,
                   legend_label=sigla,
                   color=Category20_20[i % 20])

    fig.legend.location = "top_left"
    #     fig.legend.title = "Estado"
    fig.legend.click_policy = 'hide'
    fig.add_tools(
        HoverTool(
            tooltips=[(
                "Estado",
                "@estado"), ("Dias de contaminação", f"@diasDeContaminacao_{min_casos}"),
                      ("Dia da Semana",
                       "@diaDaSemana"), (
                           "Casos Acumulados",
                           "@casosAcumulado"), ("Casos Novos", "@casosNovos"),
                      ("Óbitos Acumulados",
                       "@obitosAcumulado"), ("Óbitos Novos", "@obitosNovos")]))
    return fig


plot_casosAcumulados_por_dia_de_contaminacao_1 = get_plot_por_dia_de_contaminacao(
    "casosAcumulado", y_axis_label="Casos Acumulados", min_casos=1, width=width, height=int(1.3*height), title="Covid-19 por dia de contaminação do primeiro caso")
plot_casosAcumulados_por_dia_de_contaminacao_100 = get_plot_por_dia_de_contaminacao(
    "casosAcumulado", y_axis_label="Casos Acumulados", min_casos=100, width=width, height=int(1.3*height), title="Covid-19 por dia de contaminação do centésimo caso")
# plot_casosNovos_por_dia_de_contaminacao = get_plot_por_dia_de_contaminacao("casosNovos", y_axis_label="Casos Novos", width=width, height=height)
show(grid([[plot_casosAcumulados_por_dia_de_contaminacao_1, plot_casosAcumulados_por_dia_de_contaminacao_100]]))
# show(plot_casosAcumulados_por_dia_de_contaminacao_100)

# Verificando se estamos saindo da crise

Um bom gráfico para saber se estamos saindo da crise é o de casos novos por casos existentes (ambos em escala log). Enquanto estivermos na epidemia o gráfico aproxima-se de uma reta. Quando começarmos a sair da epidemia o gráfico cai.

In [59]:
def get_plot_casos_novos_por_casos_acumulados(df, width=1200, height=400, title="Casos novos por casos acumulados"):
    fig = figure(title=title,
                 width=width,
                 height=height,
                 x_axis_type="log",
                 y_axis_type="log",
                 x_axis_label="Casos Acumulados (log)",
                 y_axis_label="Casos Novos (log)")
    fig.line('casosAcumulado', "casosNovos", source=df)
    fig.circle('casosAcumulado', "casosNovos", source=df)
    fig.add_tools(
        HoverTool(
            tooltips=[("Dia da Semana",
                       "@diaDaSemana"), (
                           "Casos Acumulados",
                           "@casosAcumulado"), ("Casos Novos", "@casosNovos"),
                      ("Óbitos Acumulados",
                       "@obitosAcumulado"), ("Óbitos Novos", "@obitosNovos")]))
    return fig


plot_casos_novos_por_acumulados_brasil = get_plot_casos_novos_por_casos_acumulados(data_brasil, title="Casos novos por casos acumulados (Brasil)", width=width)

data_ce = data_estados[data_estados.estado=="CE"]
plot_casos_novos_por_acumulados_ce = get_plot_casos_novos_por_casos_acumulados(data_ce, title="Casos novos por casos acumulados (CE)", width=width)

show(grid([[plot_casos_novos_por_acumulados_brasil, plot_casos_novos_por_acumulados_ce]]))